In [1]:
import psycopg2 as pg2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from Player_rank import Player_ranker
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
'''
Here we're loading in all of the traditional Fantasy basketball stats into SQL, using window function to get a ranking by minutes

'''

"\nHere we're loading in all of the traditional Fantasy basketball stats into SQL, using window function to get a ranking by minutes\n\n"

In [3]:
'''IF you have open connections run the following in the psql command prompt:

SELECT pg_terminate_backend(pid) FROM pg_stat_activity WHERE datname = 'nba_capstone';

'''

"IF you have open connections run the following in the psql command prompt:\n\nSELECT pg_terminate_backend(pid) FROM pg_stat_activity WHERE datname = 'nba_capstone';\n\n"

In [4]:
conn = pg2.connect(dbname = 'postgres',host='localhost')


In [5]:
conn.autocommit = True

In [6]:
cur = conn.cursor()


In [7]:
'''Note for some reason you can not access the database if it is not all lowercase'''

cur.execute('DROP DATABASE IF EXISTS nba_capstone;')  # Makes sure there is not already a class_example database and removes is if there is
cur.execute('CREATE DATABASE nba_capstone;')



In [8]:
cur.close()
conn.close()

In [9]:
conn = pg2.connect(dbname = 'nba_capstone',host='localhost')
conn.autocommit = True

In [10]:
cur = conn.cursor()


In [11]:
query = '''
        CREATE TABLE NBA_stats (
            Season integer, 
            Player varchar(50), 
            Pos varchar(10),
            Age int,
            Tm varchar(15),
            G int,
            GS int,
            MP float,
            FG float,
            FGA float,
            FG_Percentage float,
            Threes_Made float,
            Threes_Attempted float,
            Three_Percentage float,
            Twos_Made float,
            Twos_Attempted float,
            Twos_Percentage float,
            eff_FG_Percentage float,
            FTM float,
            FTA float,
            FT_Percentage float,
            ORB float,
            DRB float,
            Rebounds float,
            AST float,
            STL float,
            BLK float,
            TOV float,
            Fouls float,
            Points float
        );
        '''

cur.execute(query)

In [12]:
# import os
# current_directory_path = os.getcwd()
# current_directory_path

query = '''
        COPY NBA_stats 
        FROM '/Users/rcheer/Desktop/Galvanize/Capstone/Fantasy-Basketball-Capstone-Project/NBA stats.csv' 
        DELIMITER ',' 
        CSV HEADER;
        '''



cur.execute(query)

In [13]:
query = '''
        CREATE TABLE nba_advanced (
            Season integer, 
            Player varchar(50), 
            Pos varchar(10),
            Age int,
            Tm varchar(15),
            G int,
            total_MP float,
            PER float,
            True_Shooting float,
            Three_Attempt_Rate float,
            FT_rate float,
            ORB_Percentage float,
            DRB_Percentage float,
            Rebound_Percentage float,
            Assist_Percentage float,
            Steal_Percentage float,
            Block_Percentage float,
            Turnover_Percentage float,
            Usage_Percentage float,
            Offensive_WinShares float,
            Defensive_WinShares float,
            WinShares float,
            WinShares_Per48 float,
            Offensive_BoxPlusMinus float,
            Defensive_BoxPlusMinus float,
            BoxPlusMinus float,
            Value_overReplacement float
        );
        '''

cur.execute(query)

In [14]:
query = '''
        COPY nba_advanced 
        FROM '/Users/rcheer/Desktop/Galvanize/Capstone/Fantasy-Basketball-Capstone-Project/NBA Advanced.csv' 
        DELIMITER ',' 
        CSV HEADER;
        '''



cur.execute(query)

In [15]:
'''
save for later
d.points,d.rebounds,d.ast,d.stl,d.blk,d.tov,d.fg_percentage,d.FT_percentage
'''


query1 = '''
            update nba_stats set Tm = 'NOP' where Tm = 'NOH';
            update nba_advanced set Tm = 'NOP' where Tm = 'NOH';
            update nba_stats set Tm = 'CHA' where Tm = 'CHO';
            update nba_advanced set Tm = 'CHA' where Tm = 'CHO';
            
            DROP TABLE IF EXISTS players;
            CREATE TABLE players AS
            select season,player,max(G) as Games from NBA_stats where Tm!='TOT' group by season,player;
            
            DROP TABLE IF EXISTS y_predictions;
            CREATE TABLE y_predictions AS
            select d.season,d.player,d.pos,d.age,MAX(case when p.player is not null then d.Tm else NULL end) as StartingTeam,SUM(G) as Games,SUM(GS) as GS,
            max(MP) as minutes
            from NBA_stats d
            left join players p
                on d.season = p.season
                and d.player = p.player
                and d.G = p.Games
            where d.Tm!='TOT'
            group by d.season,d.player,d.pos,d.age;
            
            update y_predictions set StartingTeam = 'NOP' where startingTeam = 'NOH';
            update y_predictions set StartingTeam = 'CHA' where startingTeam = 'CHO';
            
            DROP TABLE IF EXISTS rank_by_minutes;
            CREATE TABLE rank_by_minutes AS
            select y.*,n.points,n.rebounds,n.ast,n.stl,n.blk,n.tov,n.threes_made,n.fg,n.fga,n.ftm,n.fta,
            case when cast(y.GS as float)/y.Games >0.6 then 1 else 0 end as starter,
            row_number() over(partition by n.season order by MP*G desc) as min_rank from NBA_stats n
            inner join y_predictions y
                ON n.player = y.player
                and n.season = y.season
                and n.Tm=y.startingTeam;
            
            select * from rank_by_minutes        
                
        '''
cur.execute(query1)
data = cur.fetchall()
df = pd.DataFrame(np.array(data))
df.columns = ['season','player','position','age','team','gamesPlayed','gamesStarted','minutes'
              ,'points','rebounds','assists','steals','blocks','turnovers','threes_made','FGM','FGA','FTM','FTA','starter','min_rank']


In [16]:
df['min_rank']=pd.to_numeric(df['min_rank'])
df['points']=pd.to_numeric(df['points'])
df['rebounds']=pd.to_numeric(df['rebounds'])
df['assists']=pd.to_numeric(df['assists'])
df['steals']=pd.to_numeric(df['steals'])
df['blocks']=pd.to_numeric(df['blocks'])
df['turnovers']=pd.to_numeric(df['turnovers'])
df['threes_made']=pd.to_numeric(df['threes_made'])
df['FGM']=pd.to_numeric(df['FGM'])
df['FGA']=pd.to_numeric(df['FGA'])
df['FTM']=pd.to_numeric(df['FTM'])
df['FTA']=pd.to_numeric(df['FTA'])
df['gamesPlayed']=pd.to_numeric(df['gamesPlayed'])
df['gamesStarted']=pd.to_numeric(df['gamesStarted'])
df['minutes']=pd.to_numeric(df['minutes'])
df['age']=pd.to_numeric(df['age'])
df['season']=pd.to_numeric(df['season'])


In [17]:
test = Player_ranker(df)
test.get_category_dist()
test.cat_mean

,season,mean_points,mean_rebounds,mean_assists,mean_steals,mean_blocks,mean_turnovers,mean_threes_made,mean_fgm,mean_fga,mean_ftm,mean_fta
0,2008,13.3005,5.1235,2.7870,0.9150,0.5690,1.7035,0.8780,4.9095,10.5850,2.5990,3.3115
1,2009,12.9850,5.1080,2.8490,0.9120,0.5600,1.6965,0.8265,4.8530,10.4370,2.4600,3.1825
2,2010,12.9225,5.0185,2.8135,0.9085,0.5685,1.6790,0.8380,4.7955,10.3525,2.4955,3.2005
3,2011,12.1830,4.8915,2.6840,0.9225,0.5725,1.6805,0.8030,4.5925,10.1180,2.1955,2.8590
4,2012,12.5550,5.0520,2.8840,0.9585,0.6125,1.7410,0.8800,4.7285,10.3275,2.2150,2.8920
5,2013,13.0510,5.1265,2.8310,0.9505,0.5420,1.7375,0.9715,4.8545,10.5425,2.3790,3.1020
6,2014,12.3805,4.9890,2.7145,0.9230,0.5390,1.6315,0.9600,4.6260,10.1940,2.1725,2.8505
7,2015,12.8715,5.1935,2.7720,0.9530,0.5900,1.6785,1.0530,4.7645,10.4595,2.2840,2.9900
8,2016,13.3550,5.0420,2.8950,0.9230,0.5340,1.6195,1.2170,4.8980,10.6355,2.3385,2.9850
9,2017,13.2915,5.0950,2.9355,0.9240,0.5585,1.6825,1.3200,4.9175,10.6135,2.1395,2.7465


In [18]:
test.assign_values()

In [19]:
value_copy = test.value.copy()

In [20]:
value_copy.head()

,season,player,value_tot,value_points,value_rebounds,value_assists,value_blocks,value_steals,value_turnovers,value_threes,value_fg,value_ft
0,2008,Andre Iguodala,1.569149,1.022902,0.233549,1.209232,-0.313073,1.720216,-1.392654,0.164932,0.201353,-1.277308
1,2008,Joe Johnson,2.248938,1.506499,-0.293101,1.449827,-0.683573,0.464584,-1.113145,1.381647,-1.036752,0.572951
2,2008,O.J. Mayo,0.491912,0.967102,-0.536170,0.198732,-0.683573,0.464584,-1.532409,1.246456,-0.633993,1.001183
3,2008,Antawn Jamison,3.459954,1.655298,1.529918,-0.426816,-0.498323,0.715710,0.284401,0.705694,0.083254,-0.589183
4,2008,Raymond Felton,-0.135656,0.167306,-0.536170,1.882899,-0.313073,1.469090,-1.532409,-0.240639,-1.364914,0.332253


In [21]:
#copy value data back into SQL
engine = create_engine("postgresql://@localhost/nba_capstone")

value_copy.to_sql(name='value', con=engine, if_exists = 'replace', index=False)


In [22]:
query = '''
        DROP TABLE IF EXISTS player_value;
        CREATE TABLE player_value AS
        select ROW_NUMBER() OVER(PARTITION BY season ORDER BY value_tot DESC),* from value;
        
        DROP TABLE IF EXISTS value;
        
        select * from player_value;
        
        '''


cur.execute(query)
ranking_data = cur.fetchall()
df_2 = pd.DataFrame(np.array(ranking_data))
cols_value = ['playerrank']
for item in (list(value_copy.columns)):
    cols_value.append(item)
cols_value
df_2.columns=cols_value

In [23]:
for i in df_2.columns:
    if i!='player':
        df_2[i]=pd.to_numeric(df_2[i])



In [24]:
for i in df_2.columns:
    if i!='player':
        df_2[i]=pd.to_numeric(df_2[i])

In [25]:
df_2.head()

,playerrank,season,player,value_tot,value_points,value_rebounds,value_assists,value_blocks,value_steals,value_turnovers,value_threes,value_fg,value_ft
0,1,2008,Chris Paul,10.649584,1.766898,0.152526,3.952019,-0.868823,4.733733,-1.811918,-0.105449,1.195183,1.635414
1,2,2008,LeBron James,8.708530,2.808492,1.003268,2.123494,0.983678,1.971343,-1.811918,0.976075,0.888138,-0.234041
2,3,2008,Dwyane Wade,8.419529,3.143291,-0.050032,2.267852,1.354178,3.226975,-2.370936,0.300123,1.126183,-0.578103
3,4,2008,Danny Granger,6.463654,2.324895,-0.009520,-0.041864,1.539428,0.213457,-1.113145,2.463170,-0.678128,1.765360
4,5,2008,Dirk Nowitzki,6.094625,2.343495,1.327360,-0.186221,0.427928,-0.288795,-0.274618,-0.105449,0.611555,2.239370


In [26]:
df_2[df_2['season']==2017].sort_values(by='playerrank', ascending=True)

,playerrank,season,player,value_tot,value_points,value_rebounds,value_assists,value_blocks,value_steals,value_turnovers,value_threes,value_fg,value_ft
4219,1,2017,Anthony Davis,13.069292,2.728406,2.327843,-0.323373,4.382346,1.403769,-0.642579,-0.719758,2.572502,1.340136
4220,2,2017,Stephen Curry,10.977767,2.415188,0.001938,1.610250,-0.769567,1.647478,-1.635937,3.343394,1.073741,3.291282
4221,3,2017,Kevin Durant,10.555627,2.415188,0.660945,1.254056,2.665042,-0.545910,-1.635937,1.369863,1.809299,2.563082
4222,4,2017,James Harden,10.522310,3.152171,0.118234,2.984140,0.303748,2.134898,-3.374314,2.762944,-0.589500,3.029989
4223,5,2017,Karl-Anthony Towns,9.046572,1.475533,2.793024,-0.272488,1.806390,-0.302200,-0.270069,0.208962,2.213129,1.394291
4224,6,2017,LeBron James,8.050018,2.617858,1.358716,3.136795,0.733074,1.160059,-3.125975,0.557232,2.935558,-1.323300
4225,7,2017,Giannis Antetokounmpo,7.982516,2.507311,1.901427,0.948747,1.806390,1.403769,-1.635937,-0.835849,2.328774,-0.442116
4226,8,2017,Damian Lillard,7.758411,2.507311,-0.230652,1.864674,-0.340241,0.428929,-1.387598,2.066403,-0.920536,3.770119
4227,9,2017,Chris Paul,7.259998,0.978069,0.118234,2.526177,-0.769567,1.891188,-0.642579,1.369863,-0.176916,1.965529
4228,10,2017,Jimmy Butler,7.194467,1.641355,0.079468,0.999632,-0.340241,2.622318,-0.145900,-0.139308,0.324360,2.152782


In [25]:
''' Creating all teammate based changes
'''



query = '''
        DROP TABLE IF EXISTS adv_top10min;
        CREATE TEMPORARY TABLE adv_top10min as 
        select a.*,usage_percentage*total_MP/G as usage_withMins,row_number() over(partition by a.season,tm order by usage_percentage*total_MP/G desc) as usage_rank
        from(select *,row_number() over(partition by season,tm order by total_MP desc) as min_rank from nba_advanced) a
        inner join y_predictions y
                ON a.player = y.player
                and a.season = y.season
                and a.Tm=y.startingTeam
        where min_rank<=10;
        
        DROP TABLE IF EXISTS Change_Teams;
        CREATE TABLE Change_Teams AS
        select na.tm as old_team,na2.tm as new_team,na2.player,na2.pos,p.season,na.usage_withMins,
        n.points,n.rebounds,n.ast,n.threes_made,na.usage_rank 
        from player_value p
        inner join adv_top10min na
            on p.player = na.player
            and p.season = na.season+1
        inner join adv_top10min na2
            on na2.player = na.player
            and na2.season = p.season
            and na2.tm != na.tm
        inner join rank_by_minutes n
            ON N.player = na.player
            and n.season = na.season;
        
        
        
        DROP TABLE IF EXISTS incoming_by_team;
        CREATE TABLE incoming_by_team AS
        select new_team,season,SUM(case when usage_withmins >1000 then 1 else 0 end) as high_usageplayer_added,
        SUM(usage_withmins) as usagemin_added, MAX(usage_withmins) as max_usageadded,
        SUM(points) as points_added, MAX(points) as max_pointsadded,SUM(rebounds) as rebounds_added,
        MAX(rebounds) as max_reboundsadded, SUM(ast) as ast_added, MAX(ast) as max_astadded,
        SUM(threes_made) as threes_added, MAX(threes_made) as max_threesadded 
        from change_teams
        group by new_team,season;
        
        DROP TABLE IF EXISTS outgoing_by_team;
        CREATE TABLE outgoing_by_team AS
        select old_team,season,SUM(case when usage_withmins >1000 then 1 else 0 end) as high_usageplayer_dropped,
        SUM(usage_withmins) as usagemin_dropped, MAX(usage_withmins) as max_usagedropped,
        SUM(points) as points_dropped, MAX(points) as max_pointsdropped,SUM(rebounds) as rebounds_dropped,
        MAX(rebounds) as max_reboundsdropped, SUM(ast) as ast_dropped, MAX(ast) as max_astdropped,
        SUM(threes_made) as threes_dropped, MAX(threes_made) as max_threesdropped
        from change_teams
        group by old_team,season;
        
        DROP TABLE IF EXISTS Team_Changes;
        CREATE TABLE Team_Changes AS
        select c.new_team as team, c.season,c.high_usageplayer_added,o.usagemin_dropped-c.usagemin_added as usagemin_opened,
        c.max_usageadded,o.high_usageplayer_dropped,o.max_usagedropped,
        o.points_dropped-c.points_added as points_opened,max_pointsdropped,max_pointsadded,
        o.rebounds_dropped-c.rebounds_added as rebounds_opened,max_reboundsdropped,max_reboundsadded,
        o.ast_dropped-c.ast_added as ast_opened,max_astdropped,max_astadded,
        o.threes_dropped-c.threes_added as threes_opened,max_threesdropped,max_threesadded
        from incoming_by_team c
        inner join outgoing_by_team o
            ON o.old_team = c.new_team
            and o.season = c.season;
            
        DROP TABLE IF EXISTS Team_maxes;
        CREATE TABLE Team_maxes AS
        select R.season,R.startingTeam,MAX(R2.points) as pts, max(r2.rebounds) as reb, max(r2.ast) as ast,
        MAX(R2.Tov) as TO,MAX(r2.FGA) as shot_attempts, cast(NULL as float) as max_usage 
        from rank_by_minutes R
        inner join rank_by_minutes R2
            ON R2.startingTeam = R.startingTeam
            and R2.season+1 = R.season
        group by R.season,R.startingTeam;
        
        
        update Team_maxes T
        set max_usage = usage
        from
        (select T.season,T.startingTeam,T.pts,t.reb,t.ast,t.TO,MAX(a.usage_percentage) as usage from Team_maxes T
        inner join adv_top10min a
            ON a.season+1 = t.season
            and a.tm = t.startingteam
        group by T.season,T.startingTeam,T.pts,t.reb,t.ast,t.TO) A
        WHERE A.season = T.season and  A.startingTeam = T.startingTeam;
        
        
        
        '''


cur.execute(query)

In [26]:
''' Get player based data
'''



query = '''
        DROP TABLE IF EXISTS player_stats;
        CREATE TABLE player_stats AS
        select r.*,r2.starter as starter_ly,r3.points-r2.points as change_points_ly,r2.points as points_ly
        ,r3.rebounds-r2.rebounds as change_reb_ly, r2.rebounds as rebounds_ly,
        r3.ast-r2.ast as change_ast_ly, r2.ast as ast_ly,r3.stl-r2.stl as change_stl_ly,r2.stl as stl_ly,
        r3.blk-r2.blk as change_blk_ly, r2.blk as blk_ly,r3.tov-r2.tov as change_tov_ly,r2.tov as tov_ly
        from rank_by_minutes r
        left join rank_by_minutes r2
            on r.player = r2.player
            and r.season = r2.season+1
        left join rank_by_minutes r3
            ON r3.player = r2.player
            and r3.season+1 = r2.season;
        
        DROP TABLE IF EXISTS player_advstats;
        CREATE TABLE player_advstats AS
        select y.player,y.season,y.startingteam,a.per as per_ly, a2.per-a.per as change_per,a.three_attempt_rate as threeAR_ly,
        a2.three_attempt_rate-a.three_attempt_rate as change_3AR, a.rebound_percentage as reb_perc_ly, a2.rebound_percentage-a.rebound_percentage as change_reb_perc
        ,a.assist_percentage as ast_perc_ly, a2.assist_percentage-a.assist_percentage as change_assist_perc
        ,a.steal_percentage as stl_perc_ly, a2.steal_percentage-a.steal_percentage as change_stl_perc_ly
        ,a.block_percentage as blk_perc_ly, a2.block_percentage-a.block_percentage as change_blk_perc_ly
        ,a.turnover_percentage as TO_perc_ly, a2.turnover_percentage-a.turnover_percentage as change_turnover_perc_ly,
        rank() over(partition by y.season,y.startingTeam order by a.usage_percentage) as usagerank,
        rank() over(partition by y.season,a.tm order by a.usage_percentage) as usagerank_ly,
        a.offensive_winshares,
        a.defensive_winshares,a.winshares,a.winshares_per48,a.offensive_boxplusminus,a.defensive_boxplusminus,
        a.boxplusminus,a.value_overreplacement        
        from y_predictions y
        left join nba_advanced a
            ON a.player = y.player
            and a.season+1 = y.season
        left join nba_advanced a2
            ON a2.player = a.player
            and a2.season+1 = a.season;
    
        
        DROP TABLE IF EXISTS player_careerstats;
        CREATE TABLE player_careerstats AS
        select r.player,r.season,SUM(case when r2.player is not null then 1 else 0 end) as YearsPro, avg(r2.points) as career_points
        ,avg(r2.rebounds) as career_rebounds,avg(r2.ast) as career_ast, avg(r2.stl) as career_stl, avg(r2.blk) as career_blk
        ,avg(r2.tov) as career_TO, avg(r2.threes_made) as career_threesmade,avg(r2.ftm) as career_ftm,avg(r2.fta) as career_fta
        ,avg(r2.fga) as career_fga, avg(r2.fg) as career_fgm
        from rank_by_minutes r
        inner join rank_by_minutes r2
            ON r.player = r2.player
            and r.season > r2.season
        group by r.player,r.season;
       
        '''


cur.execute(query)


In [32]:
 query= '''
        
        DROP TABLE IF EXISTS points_pred;
        CREATE TABLE points_pred(
        season int, --these come from player_stats
        player varchar(50),
        Age int,
        team varchar(50),
        points float, -- these come from player_stats
        points_ly float,
        change_points_ly float,
        starter_change int,
        C_PF int,
        PG int,
        SG_SF int,
        
         -- these come from team_changes
        high_usageplayer_added int,
        usagemin_opened float,
        maxusage_added float,
        high_usageplayer_dropped int,
        points_opened float,
        max_pointsdropped float,
        max_pointsadded float,
        
        three_ar_ly float, -- from player_advstats
        change_3ar float,
        per_ly float,
        change_per float,
        usagerank float,
        usagerank_ly float,
        offensive_winshares float,
        offensive_boxplusminus float,
        boxplusminus float,
        value_overreplacement float,
        
        career_points float,
        yearspro int
        );
        
        INSERT INTO points_pred(season,player,age,team,points,points_ly,change_points_ly,starter_change,C_PF,PG,SG_SF)
        SELECT season,player,age,startingteam,points,points_ly,change_points_ly,starter-starter_ly,
        case when pos in ('C','PF') then 1 else 0 end,case when pos='PG' then 1 else 0 end,case when pos in('SG','SF') then 1 else 0 end from player_stats;
        
        update points_pred pp
        set high_usageplayer_added = tc.high_usageplayer_added,usagemin_opened=tc.usagemin_opened,
        maxusage_added=tc.max_usageadded,high_usageplayer_dropped=tc.high_usageplayer_dropped,points_opened=tc.points_opened,
        max_pointsdropped=tc.max_pointsdropped,max_pointsadded=tc.max_pointsadded
        from team_changes tc
        where tc.team = pp.team and pp.season=tc.season;
        
        update points_pred pp
        set three_ar_ly = pa.threear_ly,change_3ar=pa.change_3ar,per_ly=pa.per_ly,change_per=pa.change_per,
        usagerank=pa.usagerank,usagerank_ly=pa.usagerank_ly,offensive_winshares=pa.offensive_winshares,
        offensive_boxplusminus=pa.offensive_boxplusminus,boxplusminus=pa.boxplusminus,value_overreplacement=pa.value_overreplacement
        from player_advstats pa
        where pp.player = pa.player and pp.season = pa.season and pp.team = pa.startingteam;
        
        update points_pred pp
        set career_points = pc.career_points, yearspro = pc.yearspro
        from player_careerstats pc
        where pp.player = pc.player and pp.season = pc.season;
        
        
        select * from points_pred where season>2009
        '''


cur.execute(query)
data = cur.fetchall()
points_df = pd.DataFrame(np.array(data))
points_df.columns = ['season','player','age','team','points','points_ly','change_points_ly','starter_change','C_PF','PG','SG_SF','high_usageplayer_added','usagemin_opened','maxusage_added','high_usageplayer_dropped','points_opened','max_pointsdropped',
                    'max_pointsadded','three_ar_ly','change_3ar','per_ly','change_per','usagerank','usagerank_ly','offensive_winshares','offensive_boxplusminus','boxplusminus','value_overreplacement','career_points','yearspro']

In [33]:
rookies = points_df[points_df['points_ly'].isna()]

In [34]:
rookies.sort_values(by='points',ascending=False)

,season,player,age,team,points,points_ly,change_points_ly,starter_change,C_PF,PG,...,per_ly,change_per,usagerank,usagerank_ly,offensive_winshares,offensive_boxplusminus,boxplusminus,value_overreplacement,career_points,yearspro
416,2010,Blake Griffin,21,LAC,22.5,None,None,None,1,0,...,None,None,13,1,None,None,None,None,None,None
281,2017,Donovan Mitchell,21,UTA,20.5,None,None,None,0,0,...,None,None,15,1,None,None,None,None,None,None
181,2016,Joel Embiid,22,PHI,20.2,None,None,None,1,0,...,None,None,15,1,None,None,None,None,None,None
3845,2017,MarShon Brooks,29,MEM,20.1,None,None,None,0,0,...,None,None,21,1,None,None,None,None,8.13333,3
40,2012,Damian Lillard,22,POR,19,None,None,None,0,1,...,None,None,17,1,None,None,None,None,None,None
410,2011,Kyrie Irving,19,CLE,18.5,None,None,None,0,1,...,None,None,27,1,None,None,None,None,None,None
36,2015,Karl-Anthony Towns,20,MIN,18.3,None,None,None,1,0,...,None,None,29,1,None,None,None,None,None,None
168,2015,Jahlil Okafor,20,PHI,17.5,None,None,None,1,0,...,None,None,18,1,None,None,None,None,None,None
448,2014,Andrew Wiggins,19,MIN,16.9,None,None,None,0,0,...,None,None,21,1,None,None,None,None,None,None
81,2013,Michael Carter-Williams,22,PHI,16.7,None,None,None,0,1,...,None,None,17,1,None,None,None,None,None,None


In [35]:
players = points_df[points_df['points_ly'].notna()]

In [36]:
for i in players.columns:
    if i not in(['player','team']):
        players[i]=pd.to_numeric(players[i])
    

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
players = players.fillna(0)

In [39]:
y = players[players['season']!=2017]['points']
X = players[players['season']!=2017].drop(['points','player','season','team'],axis=1)

In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV

In [36]:
cur.close()
conn.close

<function connection.close>

In [44]:

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [39]:
r2_lr = np.mean(cross_val_score(LinearRegression(),X_train,y_train,cv=10,n_jobs=-1))
print('Linear Regression error: {}'.format(r2_lr))

r2_gb = np.mean(cross_val_score(GradientBoostingRegressor(learning_rate=0.01,n_estimators=1000,max_depth = 3,max_features=0.5),X_train,y_train,cv=5,n_jobs=-1))
print('GradientBoost error: {}'.format(r2_gb))

r2_rf = np.mean(cross_val_score(RandomForestRegressor(n_estimators=1000,max_depth = 3,max_features=0.5),X_train,y_train,cv=5,n_jobs=-1))
print('RandomForest error: {}'.format(r2_rf))

Linear Regression error: 0.7706184121337402
GradientBoost error: 0.7695143834691159
RandomForest error: 0.7068402746139153


In [65]:
parameters = {'learning_rate':[0.01,0.05,0.1], 'n_estimators':[1000,2000],'max_depth':[3,5,7,10],'max_features':[0.1,0.3,0.5]}

clf = GridSearchCV(GradientBoostingRegressor(), parameters, cv=5,verbose=3)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=1000 
[CV]  learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=1000, score=0.7495924120019375, total=   0.3s
[CV] learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=1000 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=1000, score=0.7476594944544669, total=   0.3s
[CV] learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=1000 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=1000, score=0.7943399658158287, total=   0.3s
[CV] learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=1000 
[CV]  learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=1000, score=0.7190431458804185, total=   0.3s
[CV] learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=1000 
[CV]  learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=1000, score=0.7222216933630867, total=   0.3s
[CV] learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=2000 
[CV]  learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=2000, score=0.7627053638805009, total=   0.6s
[CV] learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=2000 
[CV]  learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=2000, score=0.7502206778411044, total=   0.6s
[CV] learning_rate=0.01, max_depth=3, max_features=0.1, n_estimators=2000 
[CV]  learning_rate=0.01, max_depth=3, max_fe

[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed: 11.2min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.01, 0.05, 0.1], 'n_estimators': [1000, 2000], 'max_depth': [3, 5, 7, 10], 'max_features': [0.1, 0.3, 0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [66]:
clf.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=3, max_features=0.5,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [41]:
import keras
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Activation
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import scipy.stats as scs
import matplotlib.pyplot as plt
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [46]:
model = Sequential()
model.add(Dense(units=16,input_dim= 26,activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=4, activation='relu'))
model.add(Dense(units=1,activation='linear'))
model.compile(loss='mse', optimizer='adam')


In [47]:
model.fit(np.array(X_train), np.array(y_train), epochs=2000,batch_size=128)

Epoch 1/2000
1883/1883 [==============================] - 0s 172us/step - loss: 6281.2287
Epoch 2/2000
1883/1883 [==============================] - 0s 8us/step - loss: 2162.2463
Epoch 3/2000
1883/1883 [==============================] - 0s 9us/step - loss: 813.5049
Epoch 4/2000
1883/1883 [==============================] - 0s 8us/step - loss: 389.5246
Epoch 5/2000
1883/1883 [==============================] - 0s 8us/step - loss: 252.2155
Epoch 6/2000
1883/1883 [==============================] - 0s 9us/step - loss: 188.6177
Epoch 7/2000
1883/1883 [==============================] - 0s 8us/step - loss: 155.0123
Epoch 8/2000
1883/1883 [==============================] - 0s 9us/step - loss: 135.6868
Epoch 9/2000
1883/1883 [==============================] - 0s 9us/step - loss: 120.9900
Epoch 10/2000
1883/1883 [==============================] - 0s 8us/step - loss: 110.3256
Epoch 11/2000
1883/1883 [==============================] - 0s 11us/step - loss: 102.3767
Epoch 12/2000
1883/1883 [===========

In [48]:
predictions = model.predict(X_test)
gbr = GradientBoostingRegressor(learning_rate=0.01,n_estimators=1000,max_depth = 3,max_features=0.5)
gbr.fit(X_train,y_train)
gbr_predictions = gbr.predict(X_test)

In [49]:
len(y_test)

925

In [49]:
testing = pd.DataFrame(predictions)

In [50]:
testing.columns=['NN_predictions']

In [51]:
testing['actual'] = y_test.reset_index()['points']
testing['GBR_predictions']=gbr_predictions
testing['LY_scoring']=X_test['points_ly'].reset_index()['points_ly']

In [52]:
testing[testing['actual']>10]

,NN_predictions,actual,GBR_predictions,LY_scoring
7,15.283444,15.3,14.516847,14.2
11,14.103240,10.5,14.099980,13.3
12,11.281839,13.9,10.685212,11.5
14,15.453426,18.1,16.879166,14.3
15,12.845493,10.4,10.433422,11.3
16,6.046763,14.0,7.291565,7.0
20,14.779439,16.4,14.648856,11.7
21,11.931538,10.5,12.772061,13.5
23,13.435926,15.0,17.681255,16.9
24,3.925048,11.3,7.854545,2.0


In [151]:
X_test['points_ly'][0:20]

1019    17.8
3307     2.9
3489     7.3
3242     3.1
1007     9.4
2765     4.7
2357    18.8
3002     5.9
2866    11.0
2709    14.0
3055     6.3
1366     7.5
3278    12.8
3452     5.9
1985    12.2
2903    12.7
715     17.8
754      2.2
3450     7.2
1322    19.5
Name: points_ly, dtype: float64

In [149]:
predictions[0:20]

array([[17.844143 ],
       [ 4.58346  ],
       [10.279707 ],
       [ 4.779441 ],
       [11.738732 ],
       [ 4.8977976],
       [20.338715 ],
       [ 5.509632 ],
       [11.4254465],
       [13.024121 ],
       [ 4.4234757],
       [ 5.667042 ],
       [13.13639  ],
       [ 3.2999768],
       [12.733069 ],
       [12.993856 ],
       [17.7858   ],
       [ 3.7786436],
       [ 7.8413477],
       [16.594303 ]], dtype=float32)

0     17.877944
1      3.451549
2     10.258897
3      5.205964
4     11.953578
5      4.218924
6     18.235270
7      5.317524
8     10.236510
9     13.238273
10     5.274126
11     6.293032
12    11.519619
13     4.620900
14    14.934067
15    13.272919
16    16.929187
17     4.078159
18     8.473781
19    17.592289
dtype: float64

In [53]:
pred_2017points = players[players['season']==2017].drop(['season','team','player','points'],axis=1)

In [54]:
points_2017 = model.predict(pred_2017points)

In [55]:
test_2 =pd.DataFrame(points_2017)

In [56]:
test_3 = pd.merge(players,pred_2017points,left_index=True,right_index=True).reset_index()

In [57]:
test_3['predictions']=test_2[0]

In [59]:
test_3[['player','points','points_ly_x','predictions']].sort_values(by='points_ly_x',ascending=False)

,player,points,points_ly_x,predictions
335,Russell Westbrook,25.4,31.6,27.969471
201,James Harden,30.4,29.1,28.243748
185,Isaiah Thomas,15.6,28.9,26.031406
87,Anthony Davis,28.1,28.0,26.066639
140,DeMarcus Cousins,25.2,27.8,23.568939
139,DeMar DeRozan,23.0,27.3,26.889404
6,Damian Lillard,26.9,27.0,26.800262
264,LeBron James,27.5,26.4,25.277916
36,Kawhi Leonard,16.2,25.5,27.156918
347,Stephen Curry,26.4,25.3,25.063957


In [101]:
gbr.feature_importances_

array([0.09999954, 0.18079444, 0.05630521, 0.068424  , 0.00065053,
       0.02123672, 0.02468731, 0.00082807, 0.02697013, 0.02809122,
       0.02346993, 0.03004118, 0.04160092, 0.0432922 , 0.0426422 ,
       0.02623327, 0.04604534, 0.03923624, 0.02557597, 0.03005844,
       0.05530913, 0.0830102 , 0.00549783])

In [118]:
X_train.columns

Index(['age', 'points_ly', 'change_points_ly', 'starter_change',
       'high_usageplayer_added', 'usagemin_opened', 'maxusage_added',
       'high_usageplayer_dropped', 'points_opened', 'max_pointsdropped',
       'max_pointsadded', 'three_ar_ly', 'change_3ar', 'per_ly', 'change_per',
       'usagerank', 'usagerank_ly', 'offensive_winshares',
       'offensive_boxplusminus', 'boxplusminus', 'value_overreplacement',
       'career_points', 'yearspro'],
      dtype='object')

In [64]:
'''next is rebounds'''


query= '''
        
        DROP TABLE IF EXISTS rebounds_pred;
        CREATE TABLE rebounds_pred(
        season int, --these come from player_stats
        player varchar(50),
        Age int,
        team varchar(50),
        rebounds float, -- these come from player_stats
        rebounds_ly float,
        change_rebounds_ly float,
        C_PF int,
        PG int,
        SG_SF int,
        starter_change int, 
        
        -- these come from team_changes
        high_usageplayer_added int,
        usagemin_opened float,
        maxusage_added float,
        high_usageplayer_dropped int,
        rebounds_opened float,
        max_reboundsdropped float,
        max_reboundsadded float,
        
        -- from player_advstats
        per_ly float,
        change_per float,
        usagerank float,
        usagerank_ly float,
        reb_perc_ly float,
        change_reb_perc float,
        defensive_winshares float,
        defensive_boxplusminus float,
        boxplusminus float,
        value_overreplacement float,
        
        career_rebounds float,
        yearspro int
        );
        
        INSERT INTO rebounds_pred(season,player,age,team,rebounds,rebounds_ly,change_rebounds_ly,starter_change,C_PF,PG,SG_SF)
        SELECT season,player,age,startingteam,points,points_ly,change_points_ly,starter-starter_ly,
        case when pos in ('C','PF') then 1 else 0 end,case when pos='PG' then 1 else 0 end,case when pos in('SG','SF') then 1 else 0 end from player_stats;
        
        update rebounds_pred rp
        set high_usageplayer_added = tc.high_usageplayer_added,usagemin_opened=tc.usagemin_opened,
        maxusage_added=tc.max_usageadded,high_usageplayer_dropped=tc.high_usageplayer_dropped,rebounds_opened=tc.rebounds_opened,
        max_reboundsdropped=tc.max_reboundsdropped,max_reboundsadded=tc.max_reboundsadded
        from team_changes tc
        where tc.team = rp.team and rp.season=tc.season;
        
        update rebounds_pred rp
        set per_ly=pa.per_ly,change_per=pa.change_per,usagerank=pa.usagerank,usagerank_ly=pa.usagerank_ly
        ,reb_perc_ly = pa.reb_perc_ly,change_reb_perc = pa.change_reb_perc,defensive_winshares=pa.defensive_winshares,
        defensive_boxplusminus=pa.defensive_boxplusminus,boxplusminus=pa.boxplusminus,value_overreplacement=pa.value_overreplacement
        from player_advstats pa
        where rp.player = pa.player and rp.season = pa.season and rp.team = pa.startingteam;
        
        update rebounds_pred rp
        set career_rebounds = pc.career_rebounds, yearspro = pc.yearspro
        from player_careerstats pc
        where rp.player = pc.player and rp.season = pc.season;
        
        
        select * from rebounds_pred where season>2009
        '''


cur.execute(query)
data = cur.fetchall()
points_df = pd.DataFrame(np.array(data))
points_df.columns = ['season','player','age','team','rebounds','rebounds_ly','change_rebounds_ly','C_PF','PG','SG_SF','starter_change','high_usageplayer_added','usagemin_opened','maxusage_added','high_usageplayer_dropped','rebounds_opened','max_reboundsdropped',
                    'max_reboundsadded','per_ly','change_per','usagerank','usagerank_ly','reb_perc_ly','change_reb_perc','defensive_winshares','defensive_boxplusminus','boxplusminus','value_overreplacement','career_rebounds','yearspro']